In [1]:
# !pip install earthengine-api

In [2]:
import ee
import json
import pickle
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

### Initialize and authentication to Google Earth Engine

In [3]:
ee.Authenticate()


Successfully saved authorization token.


In [4]:
ee.Initialize()

### Get geolocation data for Kalimantan Barat

In [5]:
f = open('data/indonesia.geojson')
geojson = f.read()
f.close

<function TextIOWrapper.close()>

In [6]:
geojson = json.loads(geojson)
geojson_featureCollection = ee.FeatureCollection(geojson).filter('state == "Kalimantan Barat"')

### Initialization variable

In [7]:
initial_date = datetime.strptime('2019-01-01', '%Y-%m-%d')
final_date = datetime.strptime('2024-01-01', '%Y-%m-%d')

In [8]:
list_data = []
current_date = initial_date
error_list = []

### Getting data per day using while loop from initial date till final date

#### MOD21C3
Using data from MODIS 061 MOD21C3 (MODIS/Terra Land Surface Temperature and Emissivity Monthly L3 Global 1km SIN Grid V006) from Google Earth Engine. This dataset provides land surface temperature (LST) and emissivity data. The data is available monthly. For more information, visit this link:
https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD21C3

In [9]:
while current_date < final_date:
    print('Processing on date: ', current_date)
    
    try:
        image_collection = (
            ee.ImageCollection('MODIS/061/MOD21C3')
            .filterBounds(geojson_featureCollection)
            .filterDate(current_date)
            .select('LST_Day')
        )
    
        image_list = image_collection.getRegion(geometry=geojson_featureCollection, scale=1000).getInfo()
    
        list_data.append(image_list)
    except:
        error_list.append(current_date.strftime('%Y-%m-%d'))
        
    current_date += relativedelta(months=1)

Processing on date:  2019-01-01 00:00:00
Processing on date:  2019-02-01 00:00:00
Processing on date:  2019-03-01 00:00:00
Processing on date:  2019-04-01 00:00:00
Processing on date:  2019-05-01 00:00:00
Processing on date:  2019-06-01 00:00:00
Processing on date:  2019-07-01 00:00:00
Processing on date:  2019-08-01 00:00:00
Processing on date:  2019-09-01 00:00:00
Processing on date:  2019-10-01 00:00:00
Processing on date:  2019-11-01 00:00:00
Processing on date:  2019-12-01 00:00:00
Processing on date:  2020-01-01 00:00:00
Processing on date:  2020-02-01 00:00:00
Processing on date:  2020-03-01 00:00:00
Processing on date:  2020-04-01 00:00:00
Processing on date:  2020-05-01 00:00:00
Processing on date:  2020-06-01 00:00:00
Processing on date:  2020-07-01 00:00:00
Processing on date:  2020-08-01 00:00:00
Processing on date:  2020-09-01 00:00:00
Processing on date:  2020-10-01 00:00:00
Processing on date:  2020-11-01 00:00:00
Processing on date:  2020-12-01 00:00:00
Processing on da

### Save data to pickle

In [10]:
with open('data/raw_data_mod21c3.pkl', 'wb') as fp:
    pickle.dump(list_data, fp)

### Save data to csv

In [11]:
df = pd.DataFrame(list_data)
df.to_csv('data/raw_data_mod21c3.csv', index=False)

In [12]:
pd.DataFrame(error_list).to_csv('data/error_list_data_mod21c3.csv', index=False)